# Handwritten Digit recognizer

The purpose of this model is to develop a  Convolutional Neural Network for recognizing handwritten digits


### Import statements

In [9]:
import tensorflow as tf

from keras.preprocessing import image

from tensorflow.python.keras.callbacks import ReduceLROnPlateau

import sys
sys.path.insert(1, "/home/azureuser/DataAnalyzer/src/modules")

from modules.data_validator import *
from modules.data_processing import *

## The data preparation


#### Data visualisation

In [ ]:
#TODO:

### Data Validation

Validate the dataset using the data_validator method

In [10]:

training_path = "/home/azureuser/DataAnalyzer/digit_data/training"
testing_path =  "/home/azureuser/DataAnalyzer/digit_data/testing"
for i in range(10):
    data_validator(training_path+ str(i))
    data_validator(testing_path+ str(i))

### Generate directory to read images from & Data augmentation

#### Normalization

To avoid overfitting we will alter our dataset.
We chose :

    - Random rotation by 10° of some training image
    - Brightness_range from 30 to 70 : The user will take picture so we will adapt our model

    -Shift image to make them not in the center
        - Randomly shift images vertically
        - Randomly shift images horizontally


In [11]:
train_datagen = image.ImageDataGenerator(rescale=1.0/255.,rotation_range=10,brightness_range= [30,70], zoom_range = 0.1, # Randomly zoom image
        width_shift_range=0.3,
        height_shift_range=0.3) 
test_datagen = image.ImageDataGenerator(rescale=1.0/255.)


#### Generate and set directory to read images from with data augnemtation

In [12]:
#object of class ImageDataGenerator with the recale property
train_generator = train_datagen.flow_from_directory(training_path,batch_size=300,class_mode='sparse',target_size=(28, 28),color_mode="rgb" ,save_to_dir="/home/azureuser/digit_data/train" )

test_generator = test_datagen.flow_from_directory(testing_path, batch_size=300,class_mode='sparse',target_size=(28, 28),color_mode="rgb",save_to_dir="/home/azureuser/digit_data/test")


Found 60000 images belonging to 10 classes.
Found 10000 images belonging to 10 classes.


test_generator & train_generator are directory to read images from.
Each subdirectory (digits directory) in this directory will be considered to contain images from one class.

In [13]:
print(test_generator.image_shape)
print(test_generator.save_to_dir)
print(train_generator.class_indices)

(28, 28, 3)
/home/azureuser/digit_data/test
{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9}


## The CNN modeling and evaluation

### Model


Model : we have to avoid overfitting

Softmax :  Softmax takes a set of values, and effectively picks the biggest one, so, for example, if the output of the last layer looks like [0.1, 0.1, 0.05, 0.1, 9.5, 0.1, 0.05, 0.05, 0.05], it saves you from fishing through it looking for the biggest value, and turns it into [0,0,0,0,1,0,0,0,0] -- The goal is to save a lot of coding!
activation layer : tf.keras.layers.Dense(10, activation=tf.nn.softmax)])
10 : because we have ten classes

In [14]:

model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28,3)),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])

2022-05-31 14:59:19.041829: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/azureuser/.local/lib/python3.8/site-packages/cv2/../../lib64:
2022-05-31 14:59:19.041859: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-05-31 14:59:19.041878: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (Data-Ana): /proc/driver/nvidia/version does not exist
2022-05-31 14:59:19.042080: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


#### compile the model
Following the results got by  https://github.com/sanghvirajit/Feedforward_Neural_Network
We make the choice of the RMSprop optimizer


In [15]:
optimizer = tf.keras.optimizers.RMSprop(lr=0.001)

model.compile(optimizer = optimizer, loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

/home/azureuser/.local/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


#### Train the model

Reduce learning rate when a metric has stopped improving,we will gain in time while fitting the model by converging faster to the global minimum by decreasing the learning rate.
Set a learning rate annealer. we chose to decrease by 75%

In [16]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy',
                                            patience=5, # number of epochs with no improvement after which learning rate will be reduced.
                                            verbose=1, #update messages
                                            factor=0.25, #new_lr = lr * factor`.
                                            min_lr=0.00001 # minimum learning rate
                                            )

In [17]:
history = model.fit(train_generator, epochs=50, steps_per_epoch=50,
        validation_data=test_generator,  callbacks=[learning_rate_reduction])

Epoch 1/50
50/50 [==============================] - 23s 449ms/step - loss: 1.8902 - accuracy: 0.3381 - val_loss: 1.3084 - val_accuracy: 0.5679 - lr: 0.0010
Epoch 2/50
50/50 [==============================] - 23s 452ms/step - loss: 1.1298 - accuracy: 0.6297 - val_loss: 1.0680 - val_accuracy: 0.6360 - lr: 0.0010
Epoch 3/50
50/50 [==============================] - 25s 492ms/step - loss: 0.8368 - accuracy: 0.7341 - val_loss: 0.9332 - val_accuracy: 0.6651 - lr: 0.0010
Epoch 4/50
50/50 [==============================] - 22s 440ms/step - loss: 0.6795 - accuracy: 0.7841 - val_loss: 0.7810 - val_accuracy: 0.7145 - lr: 0.0010
Epoch 5/50
50/50 [==============================] - 23s 461ms/step - loss: 0.5504 - accuracy: 0.8250 - val_loss: 0.8238 - val_accuracy: 0.6873 - lr: 0.0010
Epoch 6/50
50/50 [==============================] - 24s 477ms/step - loss: 0.4859 - accuracy: 0.8453 - val_loss: 0.5216 - val_accuracy: 0.8253 - lr: 0.0010
Epoch 7/50
50/50 [==============================] - 23s 463ms/st

## Model Evaluation

In [ ]:
##TODO

In [19]:
evaluate_digits_model("/home/azureuser/DataAnalyzer/digit_data/validation_statique/my0.png",model)

(28, 28, 3)
1/1 [==============================] - 0s 57ms/step


AttributeError: 'numpy.ndarray' object has no attribute 'index'